# Interactive visualization
This notebook visualizes some example residues before and after the post-translational-modification.

Start with importing all necessary modules. Vienna PTM uses ``biopython Structure`` and the web application NGL for the visualization of it's Molecular Structures.

In [66]:
import nglview as nv
import os
import tempfile

from Bio.PDB import PDBIO
from copy import deepcopy

from viennaptm.dataclasses.annotatedstructure import AnnotatedStructure
from viennaptm.modification.modification.modifier import Modifier


In [67]:
def show_structure_with_highlights(structure,
                                   highlight_residues,
                                   name="structure",
                                   cartoon_color="residueindex",
                                   save_html=True):

    # to avoid side-effects, duplicate structure
    structure = deepcopy(structure)
    
    # 1. Write PDB in temp_dir
    tmp_dir = tempfile.TemporaryDirectory()

    io = PDBIO()
    io.set_structure(structure)
    pdb_file = os.path.join(tmp_dir.name, f"{name}_temp.pdb")
    io.save(pdb_file)

    # 2. Load into NGL
    view = nv.show_file(pdb_file)
    view.clear()

    # 3. Add cartoon
    view.add_representation("cartoon", color=cartoon_color)

    # 4. Highlight residues reliably
    if highlight_residues:
        for chain_id, resnum in highlight_residues:
            # robust NGL selection syntax
            sele = f"chain {chain_id} and resid {resnum}"
            view.add_representation("ball+stick", selection=":".join([str(resnum), chain_id]), colorScheme="element")

    view.center()
    
    tmp_dir.cleanup()

    # 5. Save HTML (always works)
    if save_html:
        html_file = f"{name}.html"
        nv.write_html(html_file, view)
        print(f"Saved interactive view to: {html_file}")

    return view

## Visualization of original structure

As an example application we start with visualizing the original structure of a thermostable subdomain from chicken villin headpiece (NMR). From this minimized average structure the residues: "A:50", "A:55" and "A:60" will be highlighted.

In [68]:
# import the structure from a given path
my_structure = AnnotatedStructure.from_pdb("/home/nutzer/Projects/viennaptm/tests/data/1vii.pdb")

# call the visialization method and define the residues to be highlighted
show_structure_with_highlights(
    structure=my_structure,
    highlight_residues=[("A", 50), ("A", 55), ("A", 60)],
    name="my_protein"
)

# the output visualization will be saved as html

Saved interactive view to: my_protein.html


NGLWidget()

## Modification

Modify the original residue "VAL" with chain_identifier='A' and residue_number=50 to the PTM "V3H" as well as residue "ARG" with "A:55" to "GSA". The amino accid "ASN" with "A:60" stays unchanged but will be still visualized in the next step. 

In [69]:
# generate instance of ``Modifier class``  
modifier = Modifier()

# apply "V3H" modification to the structure
modified_structure = modifier.apply_modification(structure=my_structure,
                                                 chain_identifier='A',
                                                 residue_number=50,
                                                 target_abbreviation="V3H")

# apply "GSA" modification to the structure
modified_structure = modifier.apply_modification(structure=modified_structure,
                                                 chain_identifier='A',
                                                 residue_number=55,
                                                 target_abbreviation="GSA")


/home/nutzer/miniconda3/envs/viennaptm/lib/python3.11/site-packages/Bio/PDB/Atom.py:237: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/nutzer/miniconda3/envs/viennaptm/lib/python3.11/site-packages/Bio/PDB/Atom.py:237: PDBConstructionWarning: Used element 'H' for Atom (name=H1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/nutzer/miniconda3/envs/viennaptm/lib/python3.11/site-packages/Bio/PDB/Atom.py:237: PDBConstructionWarning: Used element 'H' for Atom (name=H2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/nutzer/miniconda3/envs/viennaptm/lib/python3.11/site-packages/Bio/PDB/Atom.py:237: PDBConstructionWarning: Used element 'H' for Atom (name=H3) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/nutzer/miniconda3/envs/viennaptm/lib/python3.11/site-packages/Bio/PDB/Atom.py:237: PDBConstructionWarning: Used element 'C' for Ato

## Visualization of modified structure

The residues "A:50" and "A:55" have been modified, from "VAL" and "ARG" to "V3H" and "GSA" respectively, whereas "A:60" stayed the same ("ASN"). Now you can visualize the modified structure and compare the two structures in an interactive image.


In [70]:
# call the visialization method and define the residues to be highlighted
show_structure_with_highlights(
    structure=modified_structure,
    highlight_residues=[("A", 50), ("A", 55), ("A", 60)],
    name="my_modified_protein"
)

# keep in mind to change the name so that the output will be saved in a different html!

Saved interactive view to: my_modified_protein.html


NGLWidget()